In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style('ticks')
sns.set_color_codes()
sns.set_context('paper')

In [ ]:
hpccres = pd.read_table('../outputs/hpcc_4_1_2.tsv', sep='\t')
bwres = pd.read_table('../outputs/bw_32_1.tsv', sep='\t')

In [ ]:
hpccres.head()

In [ ]:
def label_things(data):
    data.loc[(data.i == 0) & (data.j == 1), 'nodes'] = 'neighbors'
    data.loc[(data.i == 0) & (data.j == data.j.max()), 'nodes'] = 'corners'
    data.loc[(data.i == 2) & (data.j == 3), 'nodes'] = 'even-odd'
    data['label'] = data.nodes + ' / ' + data.test
    return data

In [ ]:
hpccres = label_things(hpccres)
bwres = label_things(bwres)

In [ ]:
fg = sns.FacetGrid(hpccres, hue='label', size=10)
fg.map(plt.plot, 'm', 'time')
fg.add_legend()
plt.loglog(basex=2)

In [ ]:
sns.palplot(sns.color_palette('husl', 9))

In [ ]:
with sns.color_palette('husl', 9):
    for l in bwres.label.unique():
        d = bwres[bwres.label == l]
        linestyle = '--' if all(d.test == 'h2h') else '-'
        plt.plot(d.m, d.time, linestyle, label=l)

    plt.legend(loc=2, ncol=2)
    plt.loglog(basex=2)

In [ ]:
fg = sns.FacetGrid(bwres, hue='label', size=10)
fg.map(plt.plot, 'm', 'time')
fg.add_legend()
# plt.xlim(0, 2**12)
# plt.ylim(0, 0.00002)
# plt.loglog(basex=2)

In [ ]:
fg = sns.FacetGrid(bwres, hue='nodes', col='test', size=7/3,
                   hue_kws={'linestyle': ('-', '--', '-.'), 'marker': ('o', 's', '^'),
                            'markersize': [4] * 3})
fg.map(plt.plot, 'm', 'time')
fg.set_axis_labels('Message size [B]', 'Time [s]')
fg.add_legend()
plt.loglog(basex=2)
plt.savefig('databytest.pdf')

In [ ]:
fg = sns.FacetGrid(bwres, col='nodes', hue='test', size=6/3, 
                   hue_kws={'linestyle': ('-', '--', '-.')})#, 'marker': ('o', 's', '^')})
fg.map(plt.plot, 'm', 'time')
fg.set_axis_labels('Message size [B]', 'Time [s]')
fg.add_legend()
plt.loglog(basex=2)
plt.savefig('databynodes.pdf')

In [ ]:
from scipy.optimize import curve_fit

def line(n, s, r):
    return s + r*n 

In [ ]:
fitdata = bwres[(bwres.nodes == 'neighbors') & (bwres.test == 'ppb')]
sub1 = fitdata[(fitdata.m < 2**13) & (fitdata.m > 0)]
sub2 = fitdata[(fitdata.m >= 2**13)]

f1param, f1cov = curve_fit(line, sub1.m, sub1.time)
f2param, f2cov = curve_fit(line, sub2.m, sub2.time)

In [ ]:
plt.figure(figsize=(4, 2.8))
plt.plot(sub1.m, sub1.time, 'bo', label='Small data')
plt.plot(sub1.m, line(sub1.m, *f1param), 'b-', label='Small fit')

plt.plot(sub2.m, sub2.time, 'rs', label='Large data')
plt.plot(sub2.m, line(sub2.m, *f2param), 'r--', label='Large fit')
plt.loglog(basex=2)
sns.despine()
plt.xlim(2, 2**19)
plt.legend(loc=2)
plt.xlabel('Message size [B]')
plt.ylabel('Time [s]')
plt.tight_layout()
plt.savefig('fitresults.pdf')

In [ ]:
f1param, f2param

In [ ]:
fitdata.m.max()